In [1]:
from torch import Tensor
import torch.nn as nn

from torch.utils.tensorboard import SummaryWriter
"""
- Input image : 224 x 224 x 3
- Kernels : 11x 11 x 3
- Stride = 4
- out_channel = 96


- Input image : 224 x 224 x 3
- Kernels : 5x 5x 3
- Stride = 1
- out_channel = 256
"""
class AlexNet(nn.Module):
    def __init__(self , classes) -> None:
        super().__init__()
        self.convs = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11 , stride=4), # b * 96 * 55 * 55
        nn.LocalResponseNorm(size=5, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2), # b* 96 * 27 * 27
        nn.ReLU(),

        nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5),
        nn.LocalResponseNorm(size=5, k=2),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.ReLU(),

        nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3),
        nn.ReLU(),

        nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3),
        nn.ReLU(),

        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),  # N x 1024
    nn.Dropout(p=0.5),
    nn.Linear(in_features=(256 * 2* 2), out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=4096, out_features=4096),
    nn.ReLU(),
    nn.Linear(in_features=4096, out_features=classes),  # Adjusted for the number of classes
        )
        self.init_parameter()

    def init_parameter(self):
        for layer in self.convs:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.1),
                nn.init.constant_(layer.bias, 0)
            
        nn.init.constant_(self.convs[4].bias, 1)
        nn.init.constant_(self.convs[10].bias, 1)
        nn.init.constant_(self.convs[12].bias, 1)

    def forward(self, x:Tensor):
        x = self.convs(x)
        x = self.classifier(x)
        return x
    

In [8]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn

GPUS = [0]
EPOCHS = 90
NO_CLASSES = 1000
TRAIN_DIR = 'imagenet-mini/train'
VAL_DIR = 'imagenet-mini/val'
IMG_DIM = 227
BATCH_SIZE = 128
L_RATE = 0.005
W_DECAY = 0.0005
MOMENTUM = 0.9
CHECKPOINT_DIR = 'checkpoints/'

data_dir = "imagenet1k"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
seed = torch.initial_seed()


# create model
model = AlexNet(classes=NO_CLASSES)


# train with multi GPU
model = torch.nn.parallel.DataParallel(model, device_ids=GPUS)

# image augmentation and tranformation
data_transform = transforms.Compose([
    transforms.CenterCrop(IMG_DIM),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# parepare the dataset
imagenet_data = datasets.ImageFolder(root=data_dir, transform=data_transform)

train_size = int(0.7 * len(imagenet_data))
val_size = int(0.15 * len(imagenet_data))
test_size = len(imagenet_data) - train_size - val_size

# Split the dataset into training, validation, and test sets
train_set, val_set, test_set = random_split(imagenet_data, [train_size, val_size, test_size],generator=torch.Generator(device=device),)

train_loader = DataLoader(
    train_set,
    shuffle=False,
    batch_size=BATCH_SIZE,
    # num_workers=8
)

val_loader = DataLoader(
    val_set,
    shuffle=False,
    batch_size=BATCH_SIZE,
    # num_workers=8
)

test_loader = DataLoader(
    test_set,
    shuffle=False,
    batch_size=BATCH_SIZE,
    # num_workers=8
)

# optimizer
# optim = torch.optim.SGD(
#     model.parameters(),
#     lr=L_RATE,
#     momentum=MOMENTUM,
#     weight_decay=W_DECAY
# )

optim = torch.optim.Adam(params=model.parameters(), lr=0.0001)
# loss function
criterion = nn.CrossEntropyLoss()

# decay the learning rate
lr_scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=30, gamma=0.1)

total_steps =1
writer = SummaryWriter("tensorboard")

# training
for epoch in range(EPOCHS):
    model.train() 
    running_loss = 0.0
    for step, (X, y) in enumerate(train_loader):
        X, y = X, y
        # refreshing gradients
        optim.zero_grad()
        # forward_pass
        pred = model(X)
        # taking loss
        loss:Tensor = criterion(pred, y)
        # backward pass
        loss.backward()
        # taking step
        optim.step()
        running_loss += loss.item()
        
        
        if total_steps % 10 == 0:
            print(f'step: {total_steps} | Loss: {loss}')
            # writer.add_scalar("Loss/train", loss, step)
        total_steps += 1

        # for name, param in model.named_parameters():
        #     writer.add_histogram(name, param, epoch)
        #     writer.add_images('images', X, global_step=total_steps)

     

    # Validate the model
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{EPOCHS}, "
          f"Training Loss: {running_loss/len(train_loader):.4f}, "
          f"Validation Loss: {val_loss/len(val_loader):.4f}, "
          f"Validation Accuracy: {(correct/total) * 100:.2f}%")   
    
    # saving checkpoints
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f'model_checkpoint{epoch+1}.pkl')
    state = {
        'epoch': epoch,
        'total_steps': total_steps,
        'optimizer': optim.load_state_dict,
        'model': model.state_dict(),
        'seed': seed
    }
    writer.close()
    torch.save(state, checkpoint_path)


KeyboardInterrupt: 